In [ ]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
import datetime as dt
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.decorators import dag, task

In [ ]:
default_args = {
    'owner': 'name',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': dt.timedelta(minutes=5),
    'start_date': dt.datetime(2022, 7, 30),
}
schedule_interval = '0 11 * * *'
connection = {'host': 'host',
                      'database':'db',
                      'user':'name', 
                      'password':'password'
                     }

In [ ]:
my_token = '<token>'
bot = telegram.Bot(token=my_token)

In [ ]:
def lmk_report(chat_id): 
    q = """
    SELECT DATE(time) as day, COUNT(DISTINCT user_id) as DAU,
    sum(action='like') as likes,
    sum(action='view') as views,
    likes/views as CTR
    FROM simulator_20220720.feed_actions
    WHERE DATE(time) = DATE(yesterday())
    GROUP BY DATE(time)
    """
    df = ph.read_clickhouse(q, connection=connection)
    msg = '{}\n DAU : {:_}\n likes: {:_}\n views: {:_}\n CTR: {:.2%}'.format(df['day'][0].strftime('%d.%m.%Y'), df['DAU'][0], df['likes'][0], df['views'][0], df['CTR'][0]).replace('_', ' ')
    bot.sendMessage(chat_id=chat_id, text=msg)
    q1 = """
    SELECT DATE(time) as day, COUNT(DISTINCT user_id) as DAU,
    sum(action='like') as likes,
    sum(action='view') as views,
    likes/views as CTR
    FROM simulator_20220720.feed_actions
    WHERE DATE(time) between DATE(today()-7) and DATE(yesterday())
    GROUP BY DATE(time)
    """
    df1 = ph.read_clickhouse(q1, connection=connection)
    fig, ax = plt.subplots(2, 2, figsize=(15, 15))
    fig.suptitle('Динамика за семь дней')
    plt.subplots_adjust(hspace=0.3)
    plt.rcParams['font.size'] = '16'
    sns.lineplot(ax=ax[0, 0], data=df1, x='day', y='DAU')
    ax[0, 0].set_title('DAU')
    ax[0, 0].tick_params(axis='x', labelrotation=45)
    sns.lineplot(ax=ax[0, 1], data=df1, x='day', y='CTR')
    ax[0, 1].set_title('CTR')
    ax[0, 1].tick_params(axis='x', labelrotation=45)
    sns.lineplot(ax=ax[1, 0], data=df1, x='day', y='likes')
    ax[1, 0].set_title('likes')
    ax[1, 0].tick_params(axis='x', labelrotation=45)
    sns.lineplot(ax=ax[1, 1], data=df1, x='day', y='views')
    ax[1, 1].set_title('views')
    ax[1, 1].tick_params(axis='x', labelrotation=45)
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'Dau.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [ ]:
@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def lmk_analitic_report():
    @task()
    def make_report():
        lmk_report(chat_id='<id>')
    make_report()

In [ ]:
lmk_analitic_report = lmk_analitic_report()